# Predicting Daily Curtailment Events

The goal of this section is to motivate whether the grid state of "big curtailment" or "small/no curtailment" can be predicted just through seasonality.


## Process Overview:

1.  Label curtailment events (i.e. define a decision boundary)
2.  Partition historic data into training and test sets
3.  Fit a statistical model to the training data
4.  Predict against the test data
5.  Evaluate the performance of the model against known labels in the test data.


In [1]:
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm

from src.conf import settings

In [2]:
df = pd.concat(
    [
        pd.read_parquet(settings.DATA_DIR / f"processed/caiso/{y}.parquet") for y in range(2017,2020)
    ]
)
df.columns = df.columns.str.lower().str.replace(" ", "_")
columns = ["load", "net_load", "solar_curtailment", "solar"]
df = df[columns].groupby(pd.Grouper(freq="D")).sum()
df.reset_index(inplace=True)

/home/ttu/.local/share/virtualenvs/CaReCur-b3qbtQ7S/lib/python3.7/site-packages/fastparquet/encoding.py:222: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  Numpy8 = numba.jitclass(spec8)(NumpyIO)
/home/ttu/.local/share/virtualenvs/CaReCur-b3qbtQ7S/lib/python3.7/site-packages/fastparquet/encoding.py:224: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  Numpy32 = numba.jitclass(spec32)(NumpyIO)


In [3]:
df.head()

,timestamp,load,net_load,solar_curtailment,solar
0,2017-01-01 00:00:00+00:00,4.065994e+06,3.161397e+06,26760.716117,497133.285691
1,2017-01-02 00:00:00+00:00,6.690988e+06,5.889257e+06,43.705500,206467.221000
2,2017-01-03 00:00:00+00:00,7.231820e+06,6.737002e+06,54.841500,245951.093081
3,2017-01-04 00:00:00+00:00,7.368107e+06,6.683064e+06,20.247000,359225.032181
4,2017-01-05 00:00:00+00:00,7.245502e+06,6.373406e+06,190.026216,342741.964986


## Seasonally-driven Model

The goal of this model is to define a naive threshold that captures "significant" curtailment events.  From the EDA, we observed (very roughly) that large curtailment events could be captured by comparing a ratio of curtailment amount to solar output.  One mechanistic explanation for this could be that curtailment is most pronounced when load usage is low, but solar resource is high (e.g. temperate weather in population centers coinciding with clear sunny days.)

In [4]:
# Label Data - based on our EDA, we might start by "guessing" a threshold of importance of .05
# Later methods will be less biased, and allow for more variance.
# TODO: Try to find natural clusterings through an unsupervised process to label the dataset, and try to predict those labels.
df["curtailment_event"] = pd.Categorical(df["solar_curtailment"]/df["solar"] > .05)

df["is_weekday"] = pd.Categorical(df["timestamp"].dt.weekday.isin([5, 6]))

In [5]:
training_data = df.query("timestamp.dt.year < 2019")
test_data = df.query("timestamp.dt.year == 2019")

We hope to motivate a few basic expectations about this model.

1.  Show that seasonal variation (captured through simple time dependence) can perform better than guessing randomly.
2.  That seasonal variation alone is not sufficient to perform useful metrics

### Seasonal Effect From Load

Curtailment ~ month + weekday + net_load

In [6]:
model = "C(curtailment_event) ~ C(timestamp.dt.month) + C(is_weekday) + load"
result = smf.glm(
    model,
    training_data,
    family=sm.families.Binomial()
).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                       Generalized Linear Model Regression Results                                       
=========================================================================================================================
Dep. Variable:     ['C(curtailment_event)[False]', 'C(curtailment_event)[True]']   No. Observations:                  730
Model:                                                                       GLM   Df Residuals:                      716
Model Family:                                                           Binomial   Df Model:                           13
Link Function:                                                             logit   Scale:                          1.0000
Method:                                                                     IRLS   Log-Likelihood:                -143.47
Date:                                                           Thu, 09 Apr 2020   Deviance:                       286.94
Time:                                                                   12:27:11   Pearson chi2:                 1.10e+03
No. Iterations:                                                               24                                         
Covariance Type:                                                       nonrobust                                         
===============================================================================================
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                      -9.8119      2.464     -3.982      0.000     -14.641      -4.983
C(timestamp.dt.month)[T.2]     -0.5247      0.736     -0.713      0.476      -1.966       0.917
C(timestamp.dt.month)[T.3]     -0.7897      0.724     -1.090      0.276      -2.210       0.630
C(timestamp.dt.month)[T.4]      1.0300      0.824      1.250      0.211      -0.585       2.645
C(timestamp.dt.month)[T.5]      1.3245      0.859      1.542      0.123      -0.360       3.009
C(timestamp.dt.month)[T.6]      2.6394      1.256      2.102      0.036       0.178       5.100
C(timestamp.dt.month)[T.7]     20.8902   2.28e+04      0.001      0.999   -4.47e+04    4.48e+04
C(timestamp.dt.month)[T.8]     20.7859   2.39e+04      0.001      0.999   -4.68e+04    4.68e+04
C(timestamp.dt.month)[T.9]      1.1599      1.195      0.970      0.332      -1.183       3.503
C(timestamp.dt.month)[T.10]    -0.2105      0.756     -0.278      0.781      -1.693       1.272
C(timestamp.dt.month)[T.11]     1.2492      0.971      1.286      0.198      -0.655       3.153
C(timestamp.dt.month)[T.12]    23.0012   2.51e+04      0.001      0.999   -4.92e+04    4.92e+04
C(is_weekday)[T.True]          -0.2465      0.358     -0.688      0.491      -0.949       0.456
net_load                     2.113e-06    3.9e-07      5.423      0.000    1.35e-06    2.88e-06
===============================================================================================
"""

In [7]:
predictions = result.predict(test_data.drop(columns=["curtailment_event"]))
predictions.name = "probability"
predictions = test_data.merge(predictions, left_index=True, right_index=True)

Below is how our test data are actually labeled.

In [8]:
test_data["curtailment_event"].value_counts()

False    282
True      83
Name: curtailment_event, dtype: int64

In [9]:
predictions.query("probability > .8")["curtailment_event"].value_counts().loc[True]

44

Below is a count of our binary classification errors using an arbitrary cutoff probability of .8.  The model predicts the probability a day will have curtailment.

### Model Evaluation

We can calculate a confusion matrix and report back accuracy and precision scores.

In [10]:
true_positives = predictions.query("probability > .8")["curtailment_event"].value_counts().loc[True]
false_negatives = predictions.query("probability > .8")["curtailment_event"].value_counts().loc[False]
true_negatives = predictions.query("probability <= .8")["curtailment_event"].value_counts().loc[False]
false_positives = predictions.query("probability <= .8")["curtailment_event"].value_counts().loc[True]

accuracy = (true_positives+true_negatives)/len(predictions)
precision = true_positives / (true_positives + false_positives)
print(f"Accuracy: {accuracy}; Precision: {precision}")

Accuracy: 0.2602739726027397; Precision: 0.5301204819277109
